In [1]:
#15.433 - Finance Lab
#William Blair Team
#Getting the necessary inputs for the Option Pricing Model

In [20]:
#Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats

In [31]:
Treasury_Rate

,Date,3M Libor,1Y,3Y,5Y,7Y,10Y,15Y,30Y
0,Date,LIBOR3M,USTreasury1Y,USTreasury3Y,USTreasury5Y,USTreasury7Y,USTreasury10Y,USTreasury15Y,USTreasury30Y
1,2013-12-24,0.0024685,0.00134,0.007980000000000001,0.01724,0.0242,0.02988,0.0344,0.03894
2,2013-12-25,0.0024685,0.00134,0.00799,0.01725,0.0242,0.02989,0.0344,0.03894
3,2013-12-26,0.0024685,0.00131,0.00803,0.01736,0.02434,0.030019999999999998,0.03453,0.03924
4,2013-12-27,0.0024660000000000003,0.00126,0.007890000000000001,0.0173,0.02434,0.03011,0.034699999999999995,0.03938
...,...,...,...,...,...,...,...,...,...
2551,2020-12-17,0.0023863,9e-4,0.00185,0.00385,0.00659,0.00937,0.01142,0.016810000000000002
2552,2020-12-18,0.0023575,9.1e-4,0.00184,0.0039000000000000003,0.006670000000000001,0.00953,0.01159,0.01693
2553,2020-12-19,0.0023575,9.1e-4,0.00184,0.0039000000000000003,0.006670000000000001,0.00953,0.01159,0.01693
2554,2020-12-20,0.0023575,9.1e-4,0.00184,0.0039000000000000003,0.006670000000000001,0.00953,0.01159,0.01693


In [66]:
pd.set_option('display.max_rows', None)

In [67]:
#Reading the provided files
##Reading the Initial File
Col_Names_1 = ['Date', 'Issuer Name', 'Ticker', 'Country Code', 'Country', 'Debt Security Name', 'Bid Price USD', 'Equity Security Name', 'Equity Price USD', 'Equity Total Return', 'Fiscal Date', 'Total Assets', 'Total Liabilities', 'Total Corporate Debt', 'Total Shareholders Equity', 'Shares Outstanding', 'Dividends per Share', 'Dividends Paid', 'Dividend Yield']
Filepath_1 = 'MexicanSecurityData.csv'
#Type = {'Date':'object', 'Issuer Name':'object', 'Ticker':'object', 'Country Code':'object', 'Country':'object', 'Debt Security Name':'object', 'Bid Price USD':'float64', 'Equity Security Name':'object', 'Equity Price USD':'float64', 'Equity Total Return':'float64', 'Fiscal Date':'float64', 'Total Assets':'float64', 'Total Liabilities':'float64', 'Total Corporate Debt':'float64', 'Total Shareholders Equity':'float64', 'Shares Outstanding':'float64', 'Dividends per Share':'float64', 'Dividends Paid':'float64', 'Dividend Yield':'float64'}
Mexican_Data = pd.read_csv(Filepath_1, header = None, names = Col_Names_1, na_values = ' NA')
##Reading the Maturity File
Col_Names_2 = ['Debt Name', 'Maturity']
Filepath_2 = 'MexicanMaturities.xlsx'
Mexican_Maturities = pd.read_excel(Filepath_2, header = None, names = Col_Names_2, na_values = ' NA')
##Reading the Treasury File
Col_Names_3 = ['Date', '3M Libor', '1Y', '3Y', '5Y', '7Y', '10Y', '15Y', '30Y']
Filepath_3 = 'USTRateData.csv'
Treasury_Rate = pd.read_csv(Filepath_3, header = None, names = Col_Names_3, na_values = ' NA')
## For simplicity, we will start working with one bond - the one of Banorte
Banorte = Mexican_Data[Mexican_Data['Ticker'] == 'BANORT']
Banorte

/Users/panqian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (6,8,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Issuer Name,Ticker,Country Code,Country,Debt Security Name,Bid Price USD,Equity Security Name,Equity Price USD,Equity Total Return,Fiscal Date,Total Assets,Total Liabilities,Total Corporate Debt,Total Shareholders Equity,Shares Outstanding,Dividends per Share,Dividends Paid,Dividend Yield
1,2013-12-23,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.994969450494382,0.00209331211558883,20130930,75282791565.62529,65976126142.778465,26233571716.266262,8979457634.792517,2312837603,0.058638305953850535,32298579.2802203,0.8214482530585252
2,2013-12-24,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.942352036894696,-0.007522179184923483,20130930,75282791565.62529,65976126142.778465,26233571716.266262,8979457634.792517,2312837603,0.058638305953850535,32298579.2802203,0.825177147918512
3,2013-12-26,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.881382560879812,-0.004448225879362888,20130930,75282791565.62529,65976126142.778465,26233571716.266262,8979457634.792517,2312837603,0.058638305953850535,32298579.2802203,0.8300478895199911
4,2013-12-27,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.892352468119328,0.0015941429127745987,20130930,75282791565.62529,65976126142.778465,26233571716.266262,8979457634.792517,2312837603,0.058638305953850535,32298579.2802203,0.8282031336815201
5,2013-12-30,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.940789473684211,0.007027644884520345,20130930,75282791565.62529,65976126142.778465,26233571716.266262,8979457634.792517,2312837603,0.058638305953850535,32298579.2802203,0.8214482530585252
6,2013-12-31,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.975563385927473,0.005010080247369153,20131231,76870769594.91188,68571941223.85405,50887869403.72524,8143543543.900773,2773729563,0.05995197428448175,124746954.64280398,1.0452057793345009
7,2014-01-02,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.980288209973913,6.773394183419157e-4,20131231,76870769594.91188,68571941223.85405,50887869403.72524,8143543543.900773,2773729563,0.05995197428448175,124746954.64280398,1.0449770190413659
8,2014-01-03,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,7.051269801838298,0.010168862621311225,20131231,76870769594.91188,68571941223.85405,50887869403.72524,8143543543.900773,2773729563,0.05995197428448175,124746954.64280398,1.0351219512195122
9,2014-01-06,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.927952656568977,-0.017488643710266927,20131231,76870769594.91188,68571941223.85405,50887869403.72524,8143543543.900773,2773729563,0.05995197428448175,124746954.64280398,1.053857190155612
10,2014-01-07,MX Banorte,BANORT,MX,MEXICO,MX Banorte 5 3/4% due 31 Quoted Price Bid,NaN,GPO FIN BANORTE COM NPV SERIES O,6.983968513007583,0.008085484877772942,20131231,76870769594.91188,68571941223.85405,50887869403.72524,8143543543.900773,2773729563,0.05995197428448175,124746954.64280398,1.0500329887838136


In [4]:
#First, we get the Total MV of Assets
##Getting the necessary inputs with the correct data type
Total_Debt = pd.to_numeric(Banorte['Total Corporate Debt'], errors = 'coerce').to_numpy()
Stock_Price = pd.to_numeric(Banorte['Equity Price USD'], errors = 'coerce').to_numpy()
Shares_Outstanding = pd.to_numeric(Banorte['Shares Outstanding'], errors = 'coerce').to_numpy()
##Getting the MV of Equity
Equity = Stock_Price *  Shares_Outstanding
##Getting the MV of Assets
MV_Assets = Total_Debt + Equity
print(MV_Assets[0:7])

[4.24118001e+10 4.22901046e+10 4.21490921e+10 4.21744637e+10
 4.22864906e+10 7.02361958e+10 7.02493012e+10]


In [5]:
#Second, we get the Assets' Volatility (one year?)
##We calculate the equity volatility
Lambda = 0.9075 
Equity_Returns = pd.to_numeric(Banorte['Equity Total Return'], errors = 'coerce').to_numpy()
l = len(Equity_Returns)
Equity_Volatility = np.zeros(l)
Equity_Volatility[1] = np.sqrt(Equity_Returns[0]**2)
for i in range(2,l):
    Equity_Volatility[i] = np.sqrt(Lambda * Equity_Volatility[i-1]**2 + (1 - Lambda) * Equity_Returns[i-1]**2)
##We get the D/E ratio
DE_Ratio = Total_Debt / Equity
##Finally, we calculate the Asset Volatility
Asset_Volatility = (1 / (1 + DE_Ratio)) * Equity_Volatility
print(Asset_Volatility[0:7])

[0.         0.00079478 0.00114598 0.0012065  0.00116893 0.00099982
 0.00104136]


In [6]:
#Third, we get the strike price - straightforward
Strike_Price = Total_Debt
print(Strike_Price[0:7])

[2.62335717e+10 2.62335717e+10 2.62335717e+10 2.62335717e+10
 2.62335717e+10 5.08878694e+10 5.08878694e+10]


In [10]:
#Fourth, we get the time to maturity
##We get the maturity of the Banorte Bond from the Mexican Maturities File
Banorte_Maturity = Mexican_Maturities[Mexican_Maturities['Debt Name'] == 'MX Banorte 5 3/4% due 31 Yield to Mat Bid']
Banorte_Maturity = pd.to_datetime(Banorte_Maturity['Maturity'], format = '%Y%m%d').to_numpy()
##We get the date of the bond from the intial file
Date = pd.to_datetime(Banorte['Date'], format = '%Y/%m/%d').to_numpy()
#Finally, we take the time difference between the actual date and Maturity to get the time to expiration
Expiration_Time = ( Banorte_Maturity - Date ) / (365 * 24 * 60 * 60 * 10**9)

In [56]:
len(Expiration_Time)

1750

In [57]:
len(MV_Assets[40])

69124373064.59795

In [60]:
Asset_Volatility[30:]

array([nan, nan, nan, ..., nan, nan, nan])

In [30]:
len(Strike_Price)

1750

In [34]:
#change it later!!!
r = [1] * 1750
len(r)

1750

In [21]:
# option pricing function (no dividend)
def priceBS(S0, K, T, r, sigma, Type):
    d1 = (np.log(S0/K) + (r+0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = (np.log(S0/K) + (r-0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    if Type == "C":
        return S0*stats.norm.cdf(d1,0,1) - K*np.exp(-r*T)*stats.norm.cdf(d2, 0, 1)
    else: 
        return K*np.exp(-r*T)*stats.norm.cdf(-d2, 0, 1) - S0*stats.norm.cdf(-d1,0,1)

In [54]:
simu_asset_price = []
for i in range(1750):
    simu_asset_price.append(priceBS(MV_Assets[i], Strike_Price[i], int(Expiration_Time[i]), r[i], Asset_Volatility[i], 'C'))
simu_asset_price[30:]

/Users/panqian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/panqian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.


[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

## questions: 
### 1. which r should we use
### 2. bond volatility?

In [63]:
np.nan**2 + 1.2

nan